## Initial Wrangle 

The goal is to consolidate all of the cvs' with motion capture data into a single xarray DataSet

First, I will create a function which creates an xarray DataArray for each subject (there are 28) and each speed (there are 3). 

This array has 4 "dimensions" with 

1. Time dimension
1. "Spatial" dimension
    * Note that we also have 3 velocity "dimensions"
1. Landmark type dimension
1. Right/Left dimension

making each have shape (4500, 6, 16, 2).

Each array is named by the subject id and speed.

As it got late, I'll concatenated them into a single dataset of $28 \times 3$ data arrays, and save them as netcdf files.


In [3]:
import numpy as np
import pandas as pd
import xarray as xr
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

A quick look at one of the csvs: 

In [24]:
df = pd.read_csv('../data/raw_data/RBDS001runT35markers.txt', delimiter = '	')
df.head(3)

,Time,R.ASISX,R.ASISY,R.ASISZ,L.ASISX,L.ASISY,L.ASISZ,R.PSISX,R.PSISY,R.PSISZ,...,R.MT1Z,R.MT5X,R.MT5Y,R.MT5Z,L.MT1X,L.MT1Y,L.MT1Z,L.MT5X,L.MT5Y,L.MT5Z
0,0.000,2428.56,966.182,1175.67,2420.26,992.218,963.925,2262.56,1019.20,1122.97,...,1210.08,1935.31,272.989,1104.49,2478.43,44.5323,1114.68,2424.95,47.9302,1016.78
1,0.007,2427.46,964.619,1175.02,2418.86,990.269,964.057,2260.62,1017.43,1123.75,...,1214.32,1952.63,270.517,1108.36,2454.69,44.1712,1114.18,2400.64,47.8380,1015.82
2,0.013,2425.19,963.097,1175.69,2418.05,988.939,964.259,2258.95,1016.32,1124.15,...,1216.86,1970.66,267.118,1111.82,2430.86,43.9324,1113.85,2377.04,47.6751,1015.43


In [28]:
df.shape

(4500, 97)

That is good, as there are $1 + 2 \times 47$ rows as the paper says.

However, it turns out the time steps are not actually equally spaced, most likely due to rounding(if this project ever goes anywhere meaningful I should check this). This could make the signal processing awkward. I found it easiest to make them equally spaced by running a simple linear regression. This will give the time coordinate for the array. 

I did this using data from a single subject.

In [23]:
x = np.arange(df.shape[0])[:, None]
y = df['Time'].values

model = LinearRegression(fit_intercept = False)
model.fit(x, y, )
time_coord = model.predict(x)
print("mean absolute error of approximation", np.abs(time_coord - y).mean())

mean absolute error of approximation 0.00022223455829403023


I'll also have to change a lot of the names, separating out the left/right from the landmark portion from the x/y/z portion.

Recall that:
* x-axis is posterior to anterior
* y-axis is inferior to superior 
* z-axis is left to right 

I also changed coordinates from the "bottom-of-the-treadmill" coordinates to the midpoint of the right/left ASIS, PSIS, and illiac crest. In other words, the origin is moving in time, making the coordinates "lagrangian"

TODO: Add axis for pelvic floor

In [43]:
def make_data_array(subject, speed):
    """
    stores the positions and velocities of each landmark of a subject at a fixed speed as a 
    """
    speed = str(speed)[0] + str(speed)[-1]
    if subject < 10:
        filename = 'RBDS00' + str(subject)+ 'runT' + speed + 'markers.txt'
    if subject >= 10:
        filename = 'RBDS0' + str(subject) + 'runT' + str(speed)[0] + str(speed)[-1] + 'markers.txt'
    #load dataframe
    df = pd.read_csv('../data/raw_data/' + filename, 
                        delimiter = '	')
    # change the time coordinate
    df['Time'] = time_coord
    # set index
    df.set_index('Time', inplace = True)
    # change names 
    new_columns = [(x[2:-1], x[0], x[-1]) for x in df.columns]
    df.columns = pd.MultiIndex.from_tuples(new_columns)
    df.columns.names = ['landmark', 'side', 'axis']
    df.index.names = ['time']
    # convert to xarray
    xdf = xr.DataArray(df)
    # unstack multi-index
    xdf = xdf.unstack('dim_1')
    # compute the lift to the tangent bundle
    der = xdf.differentiate('time')
    der = der.assign_coords({'axis': ['TX', 'TY', 'TZ']})
    # combine the two 
    txdf = xr.concat([xdf, der], dim = 'axis')
    # reorder
    txdf = txdf.transpose('time', 'axis', 'landmark', 'side')
    # shift to origin to pelvic floor
    translation = (txdf.sel(landmark = 'ASIS', axis = ['X', 'Y', 'Z']).sum('side') 
                   + txdf.sel(landmark = 'PSIS', axis = ['X', 'Y', 'Z']).sum('side')
                   + txdf.sel(landmark = 'Iliac.Crest', axis = ['X', 'Y', 'Z']).sum('side'))/6
    txdf.loc[:, ['X', 'Y', 'Z'], :, :] -= translation.values[:, :, None, None]
    # add meta data
    txdf.attrs['Speed'] = speed
    txdf.attrs['Subject'] = subject
    txdf.attrs['time units'] = 'sec'
    txdf.attrs['distance units'] = 'mm'
    txdf.attrs['velocity/tangle bundle units'] = 'mm/s'
    txdf.attrs['origin of landmark positions'] = 'mid-point of the right/left ASIS/PSIS/Illiac crest'
    # give the array the obvious name:
    txdf = txdf.rename((subject, speed))
    return txdf

Here's an example output:

In [44]:
make_data_array(2, 2.5)

<xarray.DataArray (2, '25') (time: 4500, axis: 6, landmark: 16, side: 2)>
array([[[[ 9.45133333e+01,  9.33533333e+01],
         [-2.02596667e+02, -3.03226667e+02],
         [-1.69146667e+02, -2.84886667e+02],
         ...,
         [ 2.52833333e+01,  6.24533333e+01],
         [ 3.44533333e+01,  5.94233333e+01],
         [-1.08866667e+01,  1.43133333e+01]],

        [[-1.74838333e+01, -4.09608333e+01],
         [-9.60537633e+02, -7.77143833e+02],
         [-9.49582633e+02, -7.70011833e+02],
         ...,
         [-4.68285833e+02, -4.76069833e+02],
         [-3.54758833e+02, -3.64184833e+02],
         [-3.64622833e+02, -3.78724833e+02]],

        [[-1.07936000e+02,  1.08956000e+02],
         [-3.74190000e+01,  4.50560000e+01],
         [-8.26160000e+01,  8.86060000e+01],
         ...,
         [-9.98560000e+01,  1.03496000e+02],
         [-1.65443000e+02,  1.73826000e+02],
         [-1.14506000e+02,  1.22286000e+02]],

        [[-2.35500001e+02, -2.34000001e+02],
         [-2.51700001e+03,  2.21700001e+03],
         [-2.52150001e+03,  2.28600001e+03],
         ...,
         [-1.61100000e+03,  9.46500002e+02],
         [-1.12650000e+03,  7.20000002e+02],
         [-1.26450000e+03,  6.75000002e+02]],

        [[ 8.67000002e+01,  2.85900001e+02],
         [ 1.13025000e+02, -1.55550000e+03],
         [ 1.14960000e+02, -1.49685000e+03],
         ...,
         [ 1.11750000e+02,  3.65700001e+02],
         [ 9.21000002e+01,  4.03950001e+02],
         [ 2.30850001e+02,  2.33850001e+02]],

        [[ 7.12500002e+01,  1.11000000e+02],
         [ 2.76000001e+01,  4.75500001e+02],
         [ 4.87500001e+01,  4.75500001e+02],
         ...,
         [-7.56000002e+01,  2.05500001e+02],
         [ 6.43500002e+01,  1.11000000e+02],
         [-2.58000001e+01,  2.20500001e+02]]],


       [[[ 9.46650000e+01,  9.35150000e+01],
         [-2.17655000e+02, -2.86725000e+02],
         [-1.84235000e+02, -2.67925000e+02],
         ...,
         [ 1.62650000e+01,  7.04850000e+01],
         [ 2.86650000e+01,  6.59450000e+01],
         [-1.75950000e+01,  2.05350000e+01]],

        [[-1.80871667e+01, -4.02361667e+01],
         [-9.60965467e+02, -7.88695167e+02],
         [-9.49997567e+02, -7.81172167e+02],
         ...,
         [-4.68722167e+02, -4.74813167e+02],
         [-3.55326167e+02, -3.62673167e+02],
         [-3.64265167e+02, -3.78347167e+02]],

        [[-1.08118167e+02,  1.09038833e+02],
         [-3.78921667e+01,  4.75688333e+01],
         [-8.29481667e+01,  9.11188333e+01],
         ...,
         [-1.01017167e+02,  1.04208833e+02],
         [-1.65671167e+02,  1.73908833e+02],
         [-1.15335167e+02,  1.23098833e+02]],

        [[-2.20500001e+02, -2.12250001e+02],
         [-2.51250001e+03,  2.26950001e+03],
         [-2.50800001e+03,  2.30625001e+03],
         ...,
         [-1.62600000e+03,  9.27750002e+02],
         [-1.13025000e+03,  6.92250002e+02],
         [-1.25475000e+03,  6.58500002e+02]],

        [[ 1.03050000e+02,  3.34125001e+02],
         [ 1.42365000e+02, -1.59540000e+03],
         [ 1.41352500e+02, -1.49452500e+03],
         ...,
         [ 1.56150000e+02,  4.21200001e+02],
         [ 1.25550000e+02,  4.46850001e+02],
         [ 2.51475001e+02,  2.81325001e+02]],

        [[ 4.70250001e+01,  8.92500002e+01],
         [ 2.16750001e+01,  4.71750001e+02],
         [ 3.54750001e+01,  4.57500001e+02],
         ...,
         [-4.47000001e+01,  1.85250000e+02],
         [ 6.75750002e+01,  1.04250000e+02],
         [-2.96250001e+01,  1.83750000e+02]]],


       [[[ 9.46150000e+01,  9.35650000e+01],
         [-2.33055000e+02, -2.69925000e+02],
         [-1.99545000e+02, -2.51095000e+02],
         ...,
         [ 6.64500000e+00,  7.78650000e+01],
         [ 2.24250000e+01,  7.16950000e+01],
         [-2.45750000e+01,  2.61350000e+01]],

        [[-1.91063333e+01, -3.95023333e+01],
         [-9.61635933e+02, -8.01412333e+02],
         [-9.50694433e+02, -7.92935333e+02],
         ...,
         [-4.69200333e+02, -4.73450333e+02],
         [